In [1]:
import numpy as np


## Витерби

In [2]:
def viterbi(p0, x, a, e):
    L = len(x)
    V = np.zeros((2, L))
    ptr = np.zeros((L, 2))
    
    V[ :, 0] = np.log(p0) + np.log(e[:, x[0]]) 
 
    for i in range(1, L):
        for l in range(2):
            V[l, i] = np.log(e[l, x[i]]) + np.max(V[:, i - 1] + np.log(a[:, l]))
            ptr[i, l] = np.argmax(V[:, i - 1] + np.log(a[:, l]))
    
    seq = np.zeros(L, dtype=int)
 
    seq[-1] = np.argmax(V[:, -1] + np.log(a[:, -1]))
   


#идем назад 
    for i in range(L - 1, 0, -1):
        seq[i - 1] = ptr[i, seq[i]]
    return seq

## Тест 1 

In [3]:
seq1 = "ОРОРОРООРРРРРРРРРРОООООООО"
#seq1 = "ОРОРОРOOOOOO"
x = [ ]
for i in range(len(seq1)):
    if(seq1[i]=="О"):
        x.append(0)
    else:
        x.append(1)

a = np.array([[0.8, 0.2], [0.2, 0.8]])
b = np.array([[0.5, 0.5], [0.1, 0.9 ]])
p0 = (0.5, 0.5)

vit = viterbi(p0, x, a, b)
vit 

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0])

## Тест 2

In [4]:
seq1 = "ОРОРОРООРРРРРРРРРРОООООООО"

x = [ ]
for i in range(len(seq1)):
    if(seq1[i]=="О"):
        x.append(0)
    else:
        x.append(1)

a = np.array([[0.5, 0.5], [0.5, 0.5]])
b = np.array([[0.5, 0.5], [0.51, 0.49 ]])
p0 = (0.5, 0.5)

vit = viterbi(p0, x, a, b)
vit 

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1])

## FB 

In [6]:
def forward(p0, x, a, e):
    L = len(x)
    f = np.zeros((2, L))
 
    f[:, 0] = np.log(p0) + np.log(e[:, x[0]])
 
    for i in range(1, L):
        for l in range(2):
            log_term1 = f[0, i - 1] + np.log(a[0, l])
            log_term2 = np.log(1 + np.e ** (f[1, i - 1] + np.log(a[1, l]) - log_term1))
            log_sum = log_term1 + log_term2
            f[ l, i] = np.log(e[ l, x[i]]) + log_sum
 
    f_probs = f[0, -1] + np.log(1 + np.e ** (f[1, -1] - f[0, -1]))
    return f, f_probs

def backward(p0, x, a, e):
    L = len(x)
    b = np.zeros((2, L))
 
    b[:, -1] = 1
 
    for i in range(L - 2, -1, -1):
        for k in range(2):
            log_term1 = b[0, i + 1] + np.log(a[k, 0]) + np.log(e[0, x[i + 1]])
            log_term2 = np.log(1 + np.e ** (b[1, i + 1] + np.log(a[k, 1]) + np.log(e[1, x[i + 1]]) - log_term1))
            b[k, i] = log_term1 + log_term2
 
    log_term1 = b[0, 1] + np.log(p0[0]) + np.log(e[0, x[1]])
    log_term2 = np.log(1 + np.e ** (b[1, 1] + np.log(p0[1]) + np.log(e[1, x[1]]) - log_term1))
    b_probs = log_term1 + log_term2
    return b, b_probs

In [8]:
a = np.array([[0.8, 0.2], [0.2, 0.8]])
b = np.array([[0.5, 0.5], [0.1, 0.9 ]])
p0 = (0.5, 0.5)


f, f_probs = forward(p0, x, a, b)
b, b_probs = backward(p0, x, a, b)

#Вероятность состояния 
post_probs = np.e ** f * np.e ** b / np.e ** f_probs
post_probs = post_probs/np.max(post_probs)
post_probs

array([[0.8760278 , 0.78077115, 0.89514123, 0.7943244 , 0.90385713,
        0.81035542, 0.92805827, 0.87646997, 0.46574556, 0.27841736,
        0.19348522, 0.15608929, 0.14207455, 0.14238852, 0.157234  ,
        0.19620007, 0.28445594, 0.47900836, 0.90552377, 0.9830974 ,
        0.99720021, 0.99973054, 1.        , 0.99900023, 0.99305206,
        0.96026625],
       [0.14312856, 0.23838521, 0.12401513, 0.22483196, 0.11529922,
        0.20880093, 0.09109809, 0.14268639, 0.5534108 , 0.740739  ,
        0.82567113, 0.86306707, 0.87708181, 0.87676784, 0.86192236,
        0.82295628, 0.73470041, 0.540148  , 0.11363259, 0.03605896,
        0.02195615, 0.01942582, 0.01915635, 0.02015613, 0.02610429,
        0.0588901 ]])